# ***Code explanation*** 

The data is initially loaded, then cropped and augmented (data augmentation and edges augmentation). Then, the model (U-Net) is set up and trained. Finally, test data is loaded and compared with U-Net prediction.

Note: data loading and cropping in this notebook may be redundant with the notebook "preprocess_data" (which is mainly used for change detection)

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.io as tfio
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from google.colab import drive


import torchvision.transforms.functional as TF
import torchvision as tv
import torch

drive.mount('/content/drive')

### Data loading

In [ ]:
!ls drive/My\ Drive/data

In [ ]:
train_imgs = np.array([])
train_labels = np.array([])
val_imgs = np.array([])
val_labels = np.array([])
test_imgs = np.array([])
test_labels = np.array([])

In [ ]:
# load train (30 locations)
flag = False
for index_name in range(1,4):
    print("Iteration", index_name)
    if flag == False:
        train_imgs = np.load("drive/My Drive/data/train_imgs_" + str(index_name) + ".npy")
        train_labels = np.load("drive/My Drive/data/train_labels_" + str(index_name) + ".npy")
        flag = True
    else:
        train_imgs = np.concatenate([train_imgs, np.load("drive/My Drive/data/train_imgs_" + str(index_name) + ".npy")], axis=0)
        train_labels = np.concatenate([train_labels, np.load("drive/My Drive/data/train_labels_" + str(index_name) + ".npy")], axis=0)

print(train_imgs.shape)
print(train_labels.shape)


### Preprocessing

#### Random images crop

> 




In [ ]:
## Cropping - crop only one time each images

size = 200

tmp_lab = np.zeros((len(train_labels), size, size))
tmp_img = np.zeros((len(train_imgs), size, size,3))

for k in range(len(train_labels)):

  i = np.random.randint(0,len(train_labels[0])-1-size,2)

  tmp_lab[k] = train_labels[k,i[0]:i[0]+size, i[1]:i[1]+size]
  tmp_img[k] = train_imgs[k,i[0]:i[0]+size, i[1]:i[1]+size]

tmp_img = tmp_img.astype(np.uint8)
tmp_lab =tmp_lab.astype(np.uint8)

print(tmp_lab.shape)
print(tmp_img.shape)


(707, 200, 200)
(707, 200, 200, 3)



#### Data augmentation



In [ ]:
# Numbers of new images: Double the dataset

patches_imgs = tmp_img
patches_labels = tmp_lab

extand_img = patches_imgs
extand_lab = patches_labels

for k in range(len(patches_imgs)): # Transformation can be mixed

  if tf.random.uniform((),minval=0,maxval=1) < 0.25: ## Contrast : (x - mean) * contrast_factor + mean
    factor = np.random.uniform(0.4,0.9)
    mean = np.mean(patches_imgs[k])
    new_image = np.clip(mean + factor * patches_imgs[k] - factor * mean, 0, 255).astype(np.uint8)
    new_label = patches_labels[k]

  elif tf.random.uniform((),minval=0,maxval=1) < 0.25: ## flipping left rigth / np.flip(image,axis = 1)
    new_image = np.flip(patches_imgs[k],axis=1)
    new_label = np.flip(patches_labels[k],axis=1)

  elif tf.random.uniform((),minval=0,maxval=1) < 0.25: ## flipping up down / np.flip(image,axis = 0)
    new_image = np.flip(patches_imgs[k],axis=0)
    new_label = np.flip(patches_labels[k],axis=0)

  else:
    tmp = np.random.randint(1,3) ## Rotation / np.rot90(image,k=..)
    new_image = np.rot90(patches_imgs[k],k=tmp)
    new_label = np.rot90(patches_labels[k],k=tmp)


  extand_img = np.concatenate([extand_img, np.expand_dims(new_image,axis=0)],axis=0)
  extand_lab = np.concatenate([extand_lab,np.expand_dims(new_label,axis=0)],axis=0)

extand_img.dtype, extand_lab.dtype, extand_img.shape, extand_lab.shape

In [ ]:
plt.imshow(extand_img[0],cmap="gray")
plt.axis('off')

#### Countour reinforcement

In [ ]:
import scipy
from scipy.ndimage import convolve

k = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
a = convolve(tmp_img[0,:,:,0], k)
plt.imshow(a.astype(np.uint8),cmap="gray")
plt.axis('off')

### Model

In [ ]:
size = 200

In [ ]:
img_input = tf.keras.layers.Input(shape=(size, size, 3)) # classical unet
#img_input = tf.keras.layers.Input(shape=(2,img_size, img_size, 3))#shape=(img_size, img_size, 3))
#img_input_2 = tf.keras.layers.Input(shape=(img_size, img_size, 3))#shape=(img_size, img_size, 3))

#encoder
conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = layers.Dropout(0.2)(conv1)
conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2))(conv1) # classical unet
#pool1 = layers.MaxPooling3D((1, 2, 2))(conv1)


conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = layers.Dropout(0.2)(conv2)
conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = layers.MaxPooling2D((2, 2))(conv2) # classical unet
#pool2 = layers.MaxPooling3D((1, 2, 2))(conv2) # classical unet

#bottleneck
conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = layers.Dropout(0.2)(conv3)
conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

#decoder
up1 = layers.concatenate([layers.UpSampling2D((2, 2))(conv3), conv2], axis=-1)  # classical unet
#up1 = layers.concatenate([layers.UpSampling3D((1, 2, 2))(conv3), conv2], axis=-1)
conv4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = layers.Dropout(0.2)(conv4)
conv4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

up2 = layers.concatenate([layers.UpSampling2D((2, 2))(conv4), conv1], axis=-1)  # classical unet
#up2 = layers.concatenate([layers.UpSampling3D((1, 2, 2))(conv4), conv1], axis=-1)
conv5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv5 = layers.Dropout(0.2)(conv5)
conv5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)

out = layers.Conv2D( 1, (1, 1) , activation='sigmoid', padding='same')(conv5)
out = layers.Reshape((size, size, 1))(out) # classical unet
#out = layers.Reshape((2, img_size, img_size, 1))(out)

model = models.Model(inputs=img_input, outputs=out)
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy") # tester mse

In [ ]:
history = model.fit(x = extand_img[:707]/255, y = extand_lab[:707]/255, epochs=100) #validation_data=(val_imgs[:64], val_labels[:64]))

### Test

In [ ]:
# !!! LOADING BOTH TRAINING AND TEST DATA CAN CAUSE THE NOTEBOOK TO CRASH (DEPENDS ON RAM) !!!

# load test (20 locations) 
flag = False
for index_name in range(1,3):
    print("Iteration", index_name)
    if flag == False:
        test_imgs = np.load("drive/My Drive/data/test_imgs_" + str(index_name) + ".npy")
        test_labels = np.load("drive/My Drive/data/test_labels_" + str(index_name) + ".npy")
        flag = True
    else:
        test_imgs = np.concatenate([test_imgs, np.load("drive/My Drive/data/test_imgs_" + str(index_name) + ".npy")], axis=0)
        test_labels = np.concatenate([test_labels, np.load("drive/My Drive/data/test_labels_" + str(index_name) + ".npy")], axis=0)

Iteration 1
Iteration 2


In [ ]:
# Resize test images to 200x200
size = 200 

tmp_lab_test = np.zeros((len(test_labels), size, size))
tmp_img_test = np.zeros((len(test_imgs), size, size,3))

for k in range(len(test_labels)):

  i = np.random.randint(0,len(test_labels[0])-1-size,2)

  tmp_lab_test[k] = test_labels[k,i[0]:i[0]+size, i[1]:i[1]+size]
  tmp_img_test[k] = test_imgs[k,i[0]:i[0]+size, i[1]:i[1]+size]

tmp_img_test = tmp_img_test.astype(np.uint8)
tmp_lab_test =tmp_lab_test.astype(np.uint8)

print(tmp_lab_test.shape)
print(tmp_img_test.shape)


In [ ]:
#load prediction from U-net
res = model.predict(tmp_img_test)
res.shape, len

In [ ]:
# Tranform prediction from U-net to binary images
# Compared predicted with ground truth
threshold = 0.5
bin_img = np.zeros(np.array([len(res),size,size]))
Err = np.zeros(len(res))

for k in range(len(res)):
  bin_img[k][res[k,:,:,0] > threshold] = 1

for k in range(len(res)):
  for i in range(len(res[0])):
    for j in range(len(res[0])):
      Err[k] = Err[k] + np.abs(bin_img[k,i,j]-tmp_lab_test[k,i,j]/255)

ratio_err = np.mean(Err)/40000

plt.imshow(bin_img[0,:,:], cmap="gray"), bin_img.shape #0 = black

In [ ]:
plt.imshow(tmp_img_test[81],cmap="gray")
plt.axis('off')

In [ ]:
plt.imshow(tmp_lab_test[81],cmap="gray")
plt.axis('off')

In [ ]:
plt.imshow(res[81,:,:,0],cmap="gray")
plt.axis('off')